### RAG第二范式 - Advanced  RAG（朴素RAG）

<img src="../picture/img_17.png" width="50%">


1. 🔷【1】User + Query
- 这是起点，用户提出原始问题，和第一范式一样。

2. 🟦【2】Pre-Retrieval（查询前处理模块）
- ✅ 这是第二范式最显著的新增模块之一。

- 目的：让“用户原始问题”变得更适合检索（提高 Recall）

- 典型方法：

    - Query Rewriting（查询重写）：将口语化或不清晰的问题改写为更清晰明确的检索式。例如："最近AI发展怎么样" → "2023年人工智能技术发展趋势综述"

    - Query Expansion（查询扩展）：添加同义词、相关关键词，增加召回。

    - 语义结构化：将问题结构化为“主题 + 条件 + 目标”等。

- 用到的模型：T5、GPT、BART、BGE-Reranker。

3. 🟧【3】Documents + Indexing（文档与索引模块）
- 作用不变：将原始知识文档切割成 chunk，然后编码成向量，用于后续的检索。

- 区别点：

    - 第二范式中，索引过程通常不止一个维度（比如建多个索引库：语义向量库 + BM25 关键词库）。

    - 可以支持跨模态索引（如图文、图表）。

4. 🟨【4】Retrieval（检索模块）
- ✅ 这是第二范式中的“重点增强点”。

- 第一范式（Naive RAG）：

    - 仅使用一种向量检索模型（如 FAISS + SBERT）

- 第二范式的增强：

    - Hybrid Retrieval（混合检索）：同时使用 BM25（关键词）+ Dense Retrieval（语义）

    - Multi-Stage Retrieval（多阶段）：

        - Stage 1：用 Fast Dense Retriever 粗召回（如 DPR）

        - Stage 2：用 Cross Encoder 或 reranker 精排序（如 ColBERT）

- 目标：提升 召回质量 和 精确排序。

5. 🟩【5】Post-Retrieval（检索后处理模块）
- ✅ 第二范式核心中的“精炼”步骤。

- 主要任务：

    - Reranking：对候选文档重新排序，提高最相关 chunk 排在前面。

    - Context Compression：

        - 删除冗余内容，压缩上下文使其适配 LLM 输入长度限制。

        - 常见方法：TF-IDF 摘句、句向量去重、信息浓缩（如 Pegasus）。

6. ⚙️【6】Prompt 构造

- 构建 Prompt 提交给 LLM，与第一范式一致，但质量更高、上下文更相关、token 控制更智能。

7. 📩【7】Response（生成回复）
- 由 LLM（如 GPT-4）基于 Prompt 返回答案。

***总结：RAG 第二范式 = Naive RAG + 三大增强模块***

|  模块   | 增强点 | 实现方式 |
|:------:| :----: | :----: |
|  Pre-Retrieval   | 提升检索命中率 | Query Rewriting / Expansion |
|  Retrieval   | 提升召回质量 | DPR / Hybrid BM25+DPR / ColBERT |
|  Post-Retrieval   | 	提升上下文精度 | Rerank / 压缩 / 去冗余 |

#### 为什么第二范式比第一范式强？
- 因为它不仅找得更准（Pre+Post优化），还能处理更大、更复杂的上下文信息（多阶段筛选），非常适合实际部署和大规模知识库环境。

### RAG 第二范式中的 Pre-Retrieval 阶段的四个模块
|  模块名称   | 功能简述 | 是否必须 |
|:------:| :----: | :----: |
|  Query Rewriting   | 改写原始query使其更清晰 | ❌ 可选 |
|  Query Expansion   | 扩展query词汇/语义，提高召回覆盖率 | ❌ 可选 |
|  Query Classification   | 判断query类型，为路由或检索器选择提供依据 | ❌ 可选 |
|  Query Routing   | 动态选择检索器/数据源/路径 | ✅ 推荐（如果你支持多类型检索） |

模块使用与否取决于应用目标
- ✅ 如果你只是做 FAQ 问答 / 简单文档问答：
    - 可以 只保留 Query Routing 或不加 Pre-Retrieval 阶段，直接进入 Retriever → Generator 阶段即可。

- ✅ 如果你想处理多任务、多类型输入场景：
    - 推荐启用至少 Query Classification + Routing。

- ✅ 如果你的 Query 比较短（如电商检索、短问答）：
    - 建议加上 Query Expansion，提升召回。

- ✅ 如果你的用户经常提问模糊或非结构化问题：
    - 建议加上 Query Rewriting，提升稳定性与 clarity。

#### 🧠 RAG 第二范式中的 Pre-Retrieval：Query Rewriting 模块详解
1. ✅ 一、为什么需要 Query Rewriting？
- 在 Naive RAG 中，原始用户查询直接用于向量检索。如果查询：

    - 太短、太泛化（如“最新政策”）

    - 表达不明确（如“它指的是谁？”）

    - 包含错别字或语法错误

    - 语义跨度大（如“请总结最近 AI 与法律交叉的一些趋势”）

- → 都可能导致 检索失败或检索结果质量低。

- 所以，*Query Rewriting（查询重写）*模块就是用于：

    - 将“用户自然语言问题”重写为“适合用于检索的查询表达”。

2. 🧩 二、Query Rewriting 的核心目标
|  目标   | 描述 |
|:------:| :----: |
|  语义明确   | 消除歧义，如“它”改为“ChatGPT” |
|  内容扩展	   | 补全背景，例如“最新进展” → “2024年4月人工智能生成内容的发展” |
|  检索友好	   | 适配向量检索器结构（比如使用实体化概念） |

3. 🔁 三、Query Rewriting 的核心方法
- 方法 1️⃣：基于模板的重写（Template-Based）
- 思路：预定义常见问句模板，替换关键词

- 例子：
    - 原始：猫有什么特点？
    - 重写：猫的物种行为、生理特征和习性有哪些？

- 优点：可控、精细

- 缺点：覆盖有限、不智能
- 方法 2️⃣：基于语言模型的生成式重写（LLM-Based）
- 思路：利用 T5、GPT 等生成模型，将原始查询翻译为“信息需求明确”的检索式

- 典型模型：

    - T5 + 监督训练：如 CANARD 数据集

    - Prompt-Based GPT 重写：即在 Prompt 中写明重写任务，如：

        - 你是搜索引擎助手，请将以下用户自然语言问题转换成更适合检索的句子：
        - 用户问题：猫的特点是什么？
        - 输出：
    - Multi-turn QA 处理：对于多轮对话，通过模型补全上下文。
- 例子（使用 GPT）：
    - 输入：它有哪些副作用？
    - 重写：抗VEGF蛋白药物 Earicimab 的副作用有哪些？
- 方法 3️⃣：Embedding 扩展（Query Expansion）
- 思路：通过同义词库或词向量找到语义相关词，拼接进查询。

- 工具：WordNet、BERT、BM25+Embedding联合检索

- 例子：
    - 原始：猫的特征
    - 扩展：猫的行为特征、生理特性、种类差异
4. 🧱 四、Query Rewriting 的结构逻辑
```
【输入】：用户原始查询
         ↓
【Query Rewriting 模块】
    - 上下文还原（如果是多轮）
    - 语义补全（补背景词、主语）
    - 表述标准化（将“它”变为明确名词）
         ↓
【输出】：结构清晰、语义丰富的“检索查询”
```
5. 🔄 五、与第一范式的核心区别
|  模块对比   | 第一范式（Naive RAG） | 第二范式（Advanced RAG） |
|:------:| :----: | :----: |
|  是否预处理 Query   | ❌ 直接使用原始用户输入 | ✅ 加入智能预处理 |
|  检索精度   | 易失败或偏离主题 | 显著提高 Recall 与精度 |
|  对模糊查询适应性   | 差 | 好 |
|  模型交互次数   | 少 | 多（可能用 GPT 或 T5 重写） |

6. 六、研究参考与典型论文
- 📘 "Learning to Rewrite Queries for Conversational Search" (SigIR 2020, CANARD)

- 📘 "Conversational Query Reformulation" (CQR) - 用于多轮QA中的 Query Rewriting

- 📘 "FiD + DPR" 系列模型（如 FiD-KD-Rerank）

#### RAG 第二范式中 Pre-Retrieval 阶段的 Query Expansion（查询扩展）模块
1. 📘 一、什么是 Query Expansion？
- Query Expansion（查询扩展） 是指在检索前对用户原始查询语句进行增强，自动或半自动地加入一些相关词、近义词、同义短语或领域术语，以使查询表达更完整、更接近语义真值，从而提升检索到的相关文档覆盖度。

2. 🔍 二、为什么需要 Query Expansion？
- 在 RAG 第二范式中，查询内容对后续文档检索质量至关重要。Naive RAG 仅对原始 query 编码后进行向量匹配，容易错过表达不完整或语义隐含的查询。而 Query Expansion：

- 可提升召回率，防止查询过于“窄”；

- 弥补用户表达不清造成的信息缺失；

- 与 dense retriever 配合使用（如 DPR）效果尤为显著。

3. ⚙️ 三、Query Expansion 的典型方法
- 我们可以将 Query Expansion 的方法分为以下几类：

- ✅ 1. 基于知识库/同义词词典的静态扩展
- 使用 WordNet、UMLS、Domain Ontology 等；

- 将关键词扩展为其同义词、上位词、近义短语等；

- 举例：

    - 原始查询：“糖尿病治疗”

    - 扩展后：“糖尿病治疗、胰岛素控制、血糖管理、糖尿病用药”

- ✅ 2. 基于查询-文档反向索引的伪相关反馈（PRF）
- 步骤：

    - 使用原始 query 进行初步检索，得到 Top-k 初始文档；

    - 提取这些文档中高频词或 TF-IDF 高的词；

    - 将这些词添加回原 query 中作为扩展；

- 优点：无需额外训练，灵活适应不同语料；

- 缺点：对初始检索质量依赖较大。

- ✅ 3. 基于语言模型的动态扩展（当前主流）
- 使用 Encoder-Decoder 模型（如 T5、BART、GPT）；

- 输入原始 query，输出语义等价或补全后的多种 query 表达；

- 示例模型：

    - T5 for Query Expansion：通过 finetune 让模型学会“生成更完整查询”；

    - GPT for Query Brainstorming：提示词如“请为该查询扩展 3 种相关表述”。
4. 🧪 四、实际案例（如 WikiRAG 或 REALM 中）
- 例如用户输入 “谁设计了登月舱？”，系统可以扩展成：

    - “阿波罗登月舱设计者是谁？”

    - “NASA 登月计划飞船由谁制造？”

    - “登月舱的设计单位和负责人是谁？”

- 这些扩展语句输入 retriever 后，有更高概率命中包含答案的文档段落。

#### RAG 第二范式中 Pre-Retrieval 阶段的 Query Classification（查询分类）模块

1. 🧠 一、什么是 Query Classification？
- Query Classification（查询分类） 是指在检索之前，对用户的查询进行意图分类 或 语义类型识别，以便：

    - 路由到不同的索引集合（multi-index routing）；

    - 使用不同的检索策略（如规则 vs dense retrieval）；

    - 或进行更有针对性的后续处理（如多跳推理、多轮问答等）。

- 这个模块的目标是让系统“理解”查询属于什么类型，从而采用最合适的处理方式，而不是一刀切的统一处理。

2. 🎯 二、为什么需要 Query Classification？
- Naive RAG 存在“查询一律平等”的问题。实际上，不同类型的查询适合不同的处理方式。例如：
|  查询类型   | 最佳检索策略 |
|:------:| :----: |
|  实体查询	   | 精确匹配索引 / 关键词 |
|  概念解释类查询		   | dense retrieval |
|  多跳类查询		   | 需要 reasoning 模块 |
|  编程类查询	  | 检索代码语料 + 示例 |
- 通过查询分类，RAG 可以做以下优化：

    - 选择合适的向量检索模型或策略；

    - 动态选择检索参数（如top-k、embedding 模型）；

    - 启动特殊 pipeline，例如多轮问答、代码分析、图谱问答等。
3. ⚙️ 三、Query Classification 的常见方法
- ✅ 1. 规则驱动（Rule-based）
- 利用正则表达式、关键词表进行模式匹配；

- 示例：若 query 含“是什么”、“解释”、“定义”等词，则判为“定义类查询”。

- ✅ 优点：易实现、无监督
- ⚠️ 缺点：覆盖有限、无法泛化

- ✅ 2. 传统机器学习分类器（TF-IDF + SVM / XGBoost）
- 构建标注数据集：将 query 分类为 N 类（定义类、比较类、计算类…）；

- 提取文本特征（TF-IDF、POS、NER）；

- 训练分类器进行识别。

- ✅ 优点：解释性强、可调控
- ⚠️ 缺点：需要人工标注样本、对语义泛化弱

- ✅ 3. 基于预训练语言模型的语义分类（当前主流）
- 使用 BERT、RoBERTa、ERNIE 等 encoder 模型；

- 输入为 query，输出为类别标签；

- 适合迁移学习：只需少量标注即可 fine-tune；

- Huggingface 上可快速实现：
`from transformers import BertTokenizer, BertForSequenceClassification`

- ✅ 优点：语义理解好、对长查询适应强
- ⚠️ 缺点：模型推理相对慢

4. 💡 四、Query Classification 的用途和分支点
- 在 RAG 中，一旦识别出 query 类型，可以：
|  查询类型   | 行为调整 |
|:------:| :----: |
|  事实性问题	   | 调用 dense 检索器（如 DPR） |
|  实体精确查找  | 调用关键词倒排索引（如 BM25） |
|  多跳类查询		   | 触发 multi-hop reasoning 组件 |
|  冷门领域专业词	  | 走 domain-specific 索引 |
|  情感/主观问题	  | 调整 Prompt 风格或 LLM 回复策略 |
- 也可以通过 query classification 决定是否需要：

    - 使用 query rewriting

    - 使用 query expansion

    - 使用多个 retriever 并融合结果（Hybrid Retrieval）
5. 🧪 示例：Fine-tune 一个小型 BERT 分类器
- 比如，你想把查询分为如下四类：

    - definition：解释/定义型

    - comparison：比较类

    - fact：事实型

    - reasoning：多跳推理类


- 你可以构造一组小样本：
```
samples = [
    ("什么是人工智能？", "definition"),
    ("苹果和三星哪个更好？", "comparison"),
    ("中国的首都是哪里？", "fact"),
    ("如果火车晚点，我怎么才能按时到达？", "reasoning"),
]
```
- 用 BERT 进行微调后，未来所有用户 query 都可先判别类型，然后走不同的 RAG 路径。

####  RAG 第二范式中 Pre-Retrieval 阶段的最后一个模块：Query Routing（查询路由）
1. 🧠 一、什么是 Query Routing（查询路由）？
- Query Routing（查询路由） 指的是：

    - 根据用户查询的类型或语义特征，将其引导到最适合的检索路径或知识源中。

- 在 Naive RAG 中，所有查询都默认走一条“统一的 Dense 检索通道”，这在面对复杂、异构或多任务查询时效率低下。Query Routing 就是为了解决这个问题，让系统“按需分流”。

2. 🧩 二、Query Routing 的典型目标
- 选择不同类型的检索器（Retriever）

    - 比如针对事实型问题使用 Dense Retriever（如 DPR/BGE）；

    - 针对术语查找使用 BM25 倒排索引；

    - 针对图谱问答使用 SPARQL 或知识图谱接口。

- 选择合适的文档来源或索引库

    - 比如法律类查询走法条索引，医学类查询走医学库。

- 动态配置检索参数

    - 如 top-k 值、embedding 模型、是否开启 query expansion 等。

- 激活不同的后处理流程（如 Multi-hop Reasoning）

3. 📍 三、Query Routing 的工作流程
- 可以用以下结构表示整个 Query Routing 流程：
```
            ┌─────────────┐
            │ 用户原始查询 │
            └────┬────────┘
                 ↓
         ┌──────────────┐
         │ Query Classification │ ←（前置模块）
         └────┬────────────┘
              ↓
     ┌────────────────────┐
     │ Routing Decision Unit │ ← 核心模块
     └────┬────────────┬───┘
          ↓            ↓
  ┌────────────┐ ┌────────────┐
  │ DenseIndex │ │ SparseIndex│ ← 多路检索器
  └────────────┘ └────────────┘
          ↓            ↓
       合并/融合 ← （可选 Hybrid Routing）
          ↓
       下游 LLM

```

4. 🔧 四、Query Routing 的实现方式
- ✅ 1. 基于 Query 分类的路由（最常见）
- 根据前一步的 Query Classification 结果，将 query 分流：
```
if query_type == "definition":
    use_dense_retriever()
elif query_type == "exact_match":
    use_sparse_retriever()
elif query_type == "multi-hop":
    route_to_multi_hop_reasoner()
```
- ✅ 2. 基于置信度的路由
- 某些查询在 dense 检索中返回得分低、置信不足，则自动 fallback：
```
if dense_score < threshold:
    fallback_to_BM25()
```
- ✅ 3. 基于 Hybrid 检索融合（RAG-Hybrid）
- 并行调用多种检索器，融合结果（如 FAISS + BM25 + GraphRAG）：
```
dense_results = dense_retrieve(query)
bm25_results = sparse_retrieve(query)
merged = merge(dense_results, bm25_results, strategy="reciprocal_rank_fusion")
```
5. 🧠 五、Query Routing 的设计策略与分支点
|  决策点   | 可能策略 |
|:------:| :----: |
|  是否 Query Classification 后再 Routing？ | 是：更精准； 否：根据 embedding 特征直接路由 |
|  路由依据 | 分类标签、实体检测、score 区间、领域类型等 |
|  路由方式 | 单路静态、规则路由、动态路径规划、Multi-retriever |

### 🎯 一、RAG 第二范式 中 Retrieval 阶段
我们可以将其拆解为以下几类机制：
|  模块名称   | 功能说明 |
|:------:| :----: |
| Document Indexing | 构建知识库的索引结构：稀疏（BM25）、稠密（向量库）、混合 |
| Retrieval | 通过 query 获取初始 top-k 文档 |
| Re-ranking | 使用更精确的模型对检索结果重新排序，例如 cross-encoder |
| Filtering | 对内容进行去重、置信度过滤或格式标准化等 |

###### 🧩 二、Retrieval 阶段的主要方法详细讲解
1. 🔶 1. Document Indexing（文档索引构建）
- ✅ 原理
    - 将原始知识库（文本集合）转化为可快速检索的结构体，比如倒排索引或向量库。

- ✅ 常见类型
    - 稀疏索引（Sparse Indexing）：如 BM25/TF-IDF，基于词项频率的倒排索引

    - 稠密索引（Dense Indexing）：如 FAISS、Milvus，使用向量化后的嵌入表示文档

    - 混合索引（Hybrid Indexing）：将稀疏与稠密索引结合，兼具关键词匹配与语义理解能力

- ✅ 作用
    - 文档索引是整个 RAG 检索流程的**“搜索引擎后端”**，决定了后续 Retrieval 模块能否快速有效召回候选文档。

2. 🔶 2. Retrieval（文档初检召回）
- ✅ 原理
    - 根据 Query，查询前面构建的索引，返回 top-k 个与之最相关的文档段（chunk）。

- ✅ 方法
    - 对于稀疏索引：通过关键词匹配（BM25）

    - 对于稠密索引：将 query 编码成向量 → 计算与文档向量之间的余弦相似度/L2 距离 → 返回 top-k

- 查询向量计算方法：

    - 使用 BERT/SBERT

    - 使用 Dual Encoder（query 与 doc 用不同 encoder）

- ✅ 作用
    - 负责在海量文档中筛选出前 N 个最可能相关的文档。这一步的 recall rate（召回率）越高，后续回答就越可能准确。

3. 🔶 3. Re-ranking（精排）
- ✅ 原理
    - 对 Retrieval 返回的 top-k 候选文档重新打分排序，提升相关性精度。

- ✅ 方法
    - 使用更强大的语义模型进行句对评分，例如：

        - Cross-Encoder（BERT-like）：对每个 “query + doc” 输入一次模型 → 得到相似度

        - Gated Matching、ColBERT 等轻量模型

    - 使用融合打分策略（score fusion）：稀疏与稠密检索的加权平均

- ✅ 作用
    - 过滤掉相关性较低的候选文档，提高最终用于生成的文档质量，从而提升整体答案的准确率和可靠性。

4. 🔶 4. Filtering（后过滤）
- ✅ 原理
    - 在 Re-ranking 后对文档结果进行去噪和标准化，确保最终输入 LLM 的文档是“干净的、有用的”。

- ✅ 方法
    - 去重（Remove Duplicates）：避免文档重复影响生成结果

    - 格式过滤（Structure Filtering）：移除无结构、乱码、超短片段等

    - 置信度过滤（Confidence Filtering）：只保留打分高于阈值的文档

    - 领域过滤（Domain Filtering）：如果是多知识域，可做 Query→文档领域对齐

- ✅ 作用
    - 作为文档流入生成模型的“最后一道防线”，确保上下文有信息量、无冗余、无噪声，从而提升回答质量和稳定性。


### 🔄 三、与第一范式的核心区别？
|  项目   |        第一范式（Naive RAG）        | 第二范式（Advanced RAG） |
|:------:|:-----------------------------:| :----: |
|  向量检索方式  | 单一向量检索（Sentence-BERT + FAISS） | 多模态混合（BM25 + dense + reranker） |
|  检索粒度   |             单阶段召回             | 多阶段（召回 + 精排） |
|  检索行为智能性   |         静态检索器（预训练向量）          | 	可训练检索器（动态优化） |
|  是否支持复杂问题   |            一跳问题居多             | 多跳、复杂问题支持更好 |
|  查询适应能力   |          Query 直接用            | 结合 Pre-Retrieval 阶段适配后的查询策略 |

#### RAG 第二范式中的 Retrieval 阶段不再是“简单召回 top-k”，而是变成了一个可自适应、可训练、支持多模态和多阶段的智能检索系统，显著增强了对复杂问题的处理能力

### RAG 第二范式 Post-Retrieval（后检索）阶段
- 是连接检索与生成的“桥梁”，承担着对检索结果进行结构化理解、信息融合与上下文适配的任务，确保知识能够被语言模型更准确、更自然地吸收和利用。

- 这一阶段是在第一范式“简单拼接 top-k 文本”的基础上进行增强处理，其目标是：

    - ✨ 构建更有效的 Prompt，使语言模型生成结果更加准确、连贯、有根据。
- 🎯 Post-Retrieval 阶段的关键模块概览
|  模块名称   | 功能作用 |
|:------:| :----: |
| Reranking | 对从检索器返回的 Top-k 文档进行 更精细的语义排序，提高最终选中文档与用户问题的匹配度。 |
| Context Compression（上下文压缩） | 对冗长的检索结果进行摘要或提取，保留与查询最相关的信息，减少输入长度，避免超过生成模型的最大输入限制。 |
| Context Rewriting（上下文重写） | 对检索到的文本进行改写，统一语言风格，消除歧义，使内容更易于生成模型理解。 |
| Context Re-ranking（上下文重排序） | 根据与查询的相关性，对多个文档片段进行重新排序，确保最相关的信息优先被处理 |
| Context Fusion（上下文融合）	| 将多个文档片段整合成连贯的上下文，构建结构化的输入，以提高生成质量。 |
| Filtering（筛选过滤）	| 移除无关、重复或格式异常的段落，保证输入质量。 |
| Evidence Aggregation（证据聚合）| 将多个文段中的信息整合为一条连贯的证据链，支持复杂逻辑推理。|
| Answer Highlighting（答案高亮）| 将检索文段中的关键答案句、短语进行 高亮/显式标注，增强可解释性或作为引导提示。|

#### 🧩 模块详解与作用逻辑

1. 1️⃣ 重排序（Reranking）
- 核心作用：对从检索器返回的 Top-k 文档进行 更精细的语义排序，提高最终选中文档与用户问题的匹配度。

- 原理：使用双塔模型（如 bi-encoder）初步召回，然后用 Cross-Encoder 或融合注意力模型，对每个文档和查询对进行重新打分。

- 常见模型：cross-encoder/ms-marco-MiniLM-L-6-v2、BERT-Ranker、MonoT5 等。

- 适用场景：

    - 检索器召回的结果中干扰信息多。

    - 用户 query 比较模糊。

- 不适用场景：

    - 检索器（如 dense retrieval + fine-tuning）效果已经很好。

- 与第一范式区别：

    - 第一范式一般不涉及精细 reranking，仅靠向量相似度排序。



2. 🔹 模块一：Context Compression（上下文压缩）
- 🔧 做什么？
    - 对检索到的多个文档片段（Chunks）进行精简处理，只保留与用户问题相关性高的部分或使用摘要进行内容浓缩。

- 🔍 为什么要这么做？
    - 大模型存在输入长度限制（token限制），不能直接塞入过多原文

    - 检索到的内容可能冗余或无关

    - 压缩后可提升模型生成质量，降低成本

- 🛠️ 实现方式：
    - 使用 Embedding 相关度打分，只保留最相关句子

    - 使用摘要模型（如 BART、Pegasus）对长文段做 query-aware 摘要

    - 使用 token budgeting 策略做预算控制（如 LightRAG）

- 🔄 和检索阶段的衔接？
    - 它是对“粗检”文档进行“精剪”，是 Retrieval 阶段的自然收尾步骤。

3. 🔹 模块二：Context Rewriting（上下文重写）
- 🔧 做什么？
    - 对检索回来的原始片段进行 语言重写、清洗、结构优化，让其更适合被大模型理解。

- 🔍 为什么要这么做？
    - 原始文本可能是网页截取、PDF段落、带HTML符号的内容，不利于理解

    - 语言风格不统一会影响模型回答准确性

    - 可以借助 LLM 清洗上下文结构，使输入更加 简洁、连贯

- 🛠️ 实现方式：
    - 使用指令式 LLM 模型进行结构化改写（例如：“请将以下段落简化为学术风格描述...”）

    - 可以做一些标准化操作，如去除引用格式、统一语言风格

- 🎯 结果：
    - 得到的文段更 结构清晰、语言统一，提升了输入的一致性。

4. 🔹 模块三：Context Re-ranking（上下文重排序）
- 🔧 做什么？
    - 对多个检索结果根据与用户问题的相关性重新排序，确保最相关内容排前面。

- 🔍 为什么要这么做？
    - 检索模型（如 FAISS、BM25）是浅层匹配，相关性可能不准

    - 排名前的内容对生成的影响最大（尤其是在长上下文中）

- 🛠️ 实现方式：
    - 使用 Cross-Encoder 或 BERT-Reranker 模型对每个文档和问题的对拼进行打分

    - 排序后只保留前 N 个，或按权重组织上下文结构

- 📘 示例方法：
    - ColBERT

    - SPLADE-RAG

    - Sentence-BERT + CrossEncoder 组合

5. 🔹 模块四：Context Fusion（上下文融合）
- 🔧 做什么？
    - 将多个上下文片段融合成一个连贯的上下文输入，供大模型作为 prompt。

- 🔍 为什么要这么做？
    - LLM 输入是线性文本，需要将多个文档融合成一个序列

    - 不同来源的文档风格不一致，融合时要考虑 连接方式、去重、逻辑顺序

- 🛠️ 实现方式：
    - 简单拼接（加标记如【文档1】、【文档2】）

    - 分段式构造（每段前标任务，提示其用途）

    - 多文档合并再 summary 融合

- 📌 融合策略：
    - Query-aware 融合：只融合问题相关部分

    - 多文档摘要融合：通过 LLM 汇总多个段落
6. 5️⃣ 筛选过滤（Filtering）
- 核心作用：移除无关、重复或格式异常的段落，保证输入质量。

- 原理：通过规则或模型判断文本是否冗余/噪声/低质量（如 OCR 错误片段）。

- 示例：

    - 正则匹配 URL、乱码；

    - BERT 分类器判断是否与 Query 有关。

- 适用场景：

    - 使用开放式搜索引擎（如 Bing、Web Search）时。

- 不适用场景：

    - 使用结构化、已清洗的企业文档库时。

- 与第一范式区别：

    - 第一范式通常信任 retriever，不额外做清洗。
7. 6️⃣ 证据聚合（Evidence Aggregation）
- 核心作用：将多个文段中的信息整合为一条连贯的证据链，支持复杂逻辑推理。

- 原理：

    - 对多个段落抽取出相关实体/因果链/时间线，再组合。

    - 可使用 GraphQA、Retriever-Augmented Generation for Multi-hop QA 等模型。

- 适用场景：

    - 多跳问答（Who is X’s mentor’s student?）

    - 法律、医疗、科研问答等复杂知识链场景。

- 与第一范式区别：

    - 第一范式只能进行浅层组合，不具备推理能力。

8 .8️⃣ 答案高亮（Answer Highlighting）
- 核心作用：将检索文段中的关键答案句、短语进行 高亮/显式标注，增强可解释性或作为引导提示。

- 原理：

    - 用关键词匹配或 span extraction 模型（如 QA-BERT）找到关键答案；

    - 用 >>标注<<、粗体等方式在 Prompt 中高亮。

- 适用场景：

    - 需要人类阅读对齐内容（如 PDF 标注、可视化问答系统）；

    - 帮助 LLM 快速聚焦回答区域。

- 与第一范式区别：

    - 第一范式不会引导模型“关注哪一句”，是盲读。

#### 🔄 与第一范式的关键区别
### 🔍 RAG 第一范式 vs 第二范式：Post-Retrieval 阶段关键区别

| 比较维度           | 第一范式（Naive RAG）                         | 第二范式（Advanced RAG）                                  | 区别说明                                                     |
|--------------------|----------------------------------------------|-------------------------------------------------------------|--------------------------------------------------------------|
| 处理策略           | 不处理，检索结果直接拼接                    | 对检索结果进行压缩、重排、融合、改写等加工                | 是否“加工”检索内容，是两者本质区别                          |
| 上下文压缩         | ❌ 无                                         | ✅ 有（如摘要提取、关键句选择）                            | 控制token长度，提升有效信息密度                             |
| 上下文重排序       | ❌ 无                                         | ✅ 有（使用Cross-Encoder或排序模型）                      | 提升相关信息在生成中的优先级                               |
| 上下文融合         | ❌ 无                                         | ✅ 有（合并多个段落为统一上下文）                          | 提升上下文逻辑连贯性                                       |
| 上下文改写         | ❌ 无                                         | ✅ 有（用语言模型改写检索结果）                            | 使检索内容风格一致，适配生成需求                           |
| Prompt 构造方式    | 拼接原文档                                   | 拼接优化后的摘要/融合结果                                  | 提高语言模型对prompt的理解能力                             |
| 对Retriever依赖性 | 高，检索结果质量直接影响输出                | 低，可通过优化弱化无关文档影响                            | 更鲁棒，对错误检索容错性更强                               |
| 生成准确率         | 一般，易受冗余/冲突上下文影响               | 更高，输入信息经过筛选与压缩                             | 增强生成对目标问题的聚焦能力                               |
| 适用场景           | 快速原型、轻量场景                           | 精度要求高、跨域复杂问答系统                              | 二者在不同成本与精度需求下取舍                             |


#### RAG 第二范式中的 Post-Retrieval 阶段不只是“拼接文档”，而是对检索结果进行理解、组织与结构化处理，最终构造一个逻辑清晰、任务明确、信息准确的 Prompt，从而显著提升语言模型生成质量与可控性。

| 模块名称                | 是否必须 | 主要作用                                                         | 适用场景                                                                 |
|-------------------------|----------|------------------------------------------------------------------|--------------------------------------------------------------------------|
| 重排序 Reranking         | ❌ 否     | 使用 Cross-Encoder 或打分模型对 Top-k 检索结果重新排序           | 检索器性能有限，或存在冗余、不相关文段时                                 |
| 上下文压缩 Compression    | ❌ 否     | 选择最相关片段，提取关键句，减少输入 Token 长度                  | 召回冗余多、LLM 输入限制或回答需精准时                                   |
| 上下文融合 Fusion         | ❌ 否     | 将多个检索结果融合为统一上下文（例如多段合成摘要）               | 回答需要整合多个片段（如多跳问答、摘要、报告类输出）                    |
| 上下文改写 Rewriting      | ❌ 否     | 改写文档片段，使其语义统一、风格一致、逻辑更连贯                 | 文档风格混杂，结构性不强，或 LLM 对原文理解不佳时                        |
| 筛选过滤 Filtering        | ❌ 否     | 对不相关、噪声或低质量文档进行过滤处理                           | Web 检索、开源数据源中噪声较多时                                         |
| 证据聚合 Evidence Aggregation | ❌ 否     | 多个片段联合支撑复杂问答，形成推理链                              | 多跳问答、因果关系推理、科学问答等需要综合多个证据的任务                |
| 答案高亮 Answer Highlighting | ❌ 否     | 预标答案位置，便于人类阅读/解释或提供更强引导                     | 面向用户展示/解释，或增强生成式问答一致性（例如 PDF/网页问答）          |

- 在实际实现中建议根据下列目标动态选择：

    - ✅ 精准回答 → Reranking + Compression

    - ✅ 多段融合 → Fusion + Aggregation

    - ✅ 复杂推理 → Rewriting + Aggregation

    - ✅ 对用户友好展示 → Highlighting + Rewriting

    - ✅ 清洗原始搜索结果 → Filtering


## RAG搭建
- ✅ 文件结构总览
# ✅ RAG 第二范式完整实现推荐文件结构
```
rag_v2_project/
├── indexing/                        # 📦 构建向量索引（先验阶段）
│   ├── document_splitter.py        # 文本分块器模块（支持 chunk size, overlap）
│   ├── embedding_generator.py      # 文本向量化模块（支持多模型）
│   ├── index_builder.py            # 构建稠密 / 稀疏 / 混合索引
│   ├── index_saver_loader.py       # 索引保存与加载逻辑
│   └── test_indexing.py            # 单元测试用例
│
├── pre_retrieval/                  # 🔁 检索前阶段（Query处理）
│   ├── query_rewriting.py          # Query Rewriting（语义重写）
│   ├── query_expansion.py          # Query Expansion（近义扩展）
│   ├── query_classification.py     # Query Classification（分类路由）
│   ├── query_routing.py            # Query Routing（路由目标检索器）
│   └── test_pre_retrieval.py       # 单元测试
│
├── retrieval/                      # 🔍 检索阶段
│   ├── dense_retriever.py          # 基于向量相似度的稠密检索器（如 Faiss）
│   ├── sparse_retriever.py         # 稀疏检索（如 BM25）
│   ├── hybrid_retriever.py         # 混合检索逻辑（稠密 + 稀疏融合）
│   ├── retriever_router.py         # 检索器选择与融合接口
│   └── test_retrieval.py           # 单元测试
│
├── post_retrieval/                 # 🔧 检索后优化阶段
│   ├── context_compression.py      # 上下文压缩（摘要/去冗）
│   ├── context_reranking.py        # Cross-Encoder 重排序模块
│   ├── context_fusion.py           # 逻辑拼接/多跳融合
│   ├── context_rewriting.py        # 上下文风格/语义改写
│   └── test_post_retrieval.py      # 单元测试
│
├── generation/                    # 🧠 大模型生成阶段
│   ├── prompt_builder.py           # 构造统一的 Prompt 格式
│   ├── llm_interface.py            # 接入 OpenAI, Claude, ChatGLM 等模型
│   ├── response_parser.py          # 提取、验证、解释生成内容
│   └── test_generation.py          # 单元测试
│
├── pipeline/                      # 🧪 总控管道，集成全流程调用
│   ├── rag_pipeline.py             # 主流程调用类（集成所有模块）
│   └── test_pipeline.py            # 流程级测试
│
├── utils/                         # 🧰 通用工具函数
│   ├── config.py                   # 超参数与路径配置
│   ├── logging_utils.py            # 日志打印与保存
│   └── evaluation_metrics.py       # 支持 EM, BLEU, Rouge 等指标
│
├── scripts/                       # 🔧 CLI & 启动脚本
│   ├── build_index.py              # 单独构建索引
│   ├── run_pipeline.py             # 主流程调用脚本
│   └── demo_query.py               # 测试交互入口
│
└── data/                          # 📁 存放原始文档与索引文件
    ├── raw_docs/                   # 原始知识文件夹
    ├── vector_store/               # Faiss/Elasticsearch索引存储
    └── logs/                       # 流程运行日志
```